In [1]:
import syllables 
import pandas as pd
import random
import numpy as np
import re
import language_tool_python
from IPython.display import display
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import timeit

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jennifergutman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import language_tool_python

In [3]:
tool = language_tool_python.LanguageTool('en-US')

In [4]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jennifergutman/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
# !pip install gingerit

In [6]:
with open("Dataset_poems.txt", "r",encoding="utf8") as file:
    # Trial_data.txt
    # Dataset_poems.txt
    temp = []
    for line in file:
        line1 = re.sub('[^a-zA-Z ]', '', line)
        line1 = line1.split()
        if line1 != ['', ''] and line1 != ['']:
            for i in line1:
                if i == '':
                    continue
                else:
                    temp.append(i.lower())
words_df = temp[:5000] 

In [7]:
tokens_tag = pos_tag(words_df)
dic_tag = {}
for tup in tokens_tag:
  if tup[1] not in dic_tag.keys():
    dic_tag[tup[1]] = [tup[0]]
  else:
    temp = dic_tag[tup[1]] + [tup[0]]
    dic_tag[tup[1]] = temp
# print(dic_tag.keys())
z = list(dic_tag.values())
words_df2 = []
for i in range(len(z)):
  words_df2.extend(z[i])
# print(words_df2)
words_df = words_df2

In [8]:
lines = []
for i in range(3, len(words_df)-3):
    lines.append(words_df[i-3]+' '+' '+words_df[i-2]+' '+' '+words_df[i-1]+' '+' '+words_df[i]+' '+' '+words_df[i+1]+' '+' '+words_df[i+2]+' '+' '+words_df[i+3])

# sklearn countvectorizer
from sklearn.feature_extraction.text import CountVectorizer
# Convert a collection of text documents to a matrix of token counts
cv = CountVectorizer(ngram_range=(1,1), stop_words = stopwords.words('english'))
# matrix of token counts
X = cv.fit_transform(lines)
Xc = (X.T * X) # matrix manipulation
Xc.setdiag(0) # set the diagonals to be zeroes as it's pointless to be 1

names = cv.get_feature_names() # This are the entity names (i.e. keywords)
cooccur_df = pd.DataFrame(data = Xc.toarray(), columns = names, index = names)
total = cooccur_df.to_numpy().sum()
cooccur_df = cooccur_df.div(total)
cooccur_df.to_csv('co_occur_mat.csv', sep = ',')

In [9]:
class poem_generation:
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)

    def select_random_words(self,words_df):
        words_array1 = []
        while len(words_array1) <= 15:
            random_number = random.randint(0, len(words_df)-1)
            if words_df[random_number] != 0:
                words_array1.append(words_df[random_number])
        # print(len(words_array1))

        return words_array1

    def create_poems(self,arr):
        poem = [0 for _ in range(4)]
        for i in range(1,5):
            temp =  arr[4*(i-1):4*i]
            poem[i-1] = temp
            # print("HERE:",temp)
            # print(type(temp))
            poem[i-1]=' '.join(poem[i-1])
            # print(poem[i-1])
        # print("=========================================================")
        return poem

    def store_syllables(self,arr):
        a = [0 for _ in range(4)]
        temp = 0
        j = 0
        for words,i in zip(arr,range(0,16)):
            temp = temp + syllables.estimate(words)
            if i in [3,7,11,15]:
                a[j] = temp
                # print(a[j])
                temp = 0
                j += 1
        return a

    def Grammar_checker(self,text):
        error = 0
        poem_error = []
        # tool = language_tool_python.LanguageTool('en-US')
        for lines in text:
            matches = tool.check(lines)
            if len(matches)>0:
                error = error + matches[0].errorLength
        poem_error.append(error)
        return poem_error

    def crossover(self,p1,p2,cross):
        part1 = []
        part2 = []
        part3 = []
        child1 = []
        child2 = []
        j = 0
        k = 0
        random_number1 = sorted([random.randint(1, 15) for i in range(cross)])
        random_number2 = sorted([random.randint(1, 15) for i in range(cross)])
        for i in random_number1:
            part1.append(p1[j:i])
            j = i
        for i in random_number2:
            part2.append(p2[k:i])
            k = i
        part1.append(p1[j:])
        part2.append(p2[k:])
        part2.extend(part1)
        random.shuffle(part2)
        for i in range(len(part2)):
            part3.extend(part2[i])
        for i in range(int(len(part3)/2)):
            child1.append(part3[i])
        for i in range(int(len(part3)/2),int(len(part3))):
            child2.append(part3[i])
        # if child1 == p1 or child1 == p2 or child2 == p1 or child2 == p2:
        #     child1, child2,child1_poem,child2_poem = self.crossover(child1,child2,cross)
        child1_poem = self.create_poems(child1)
        child2_poem = self.create_poems(child2)
        return child1,child2,child1_poem,child2_poem
    
    def coherence_checker(self, text):
        clean_text = ''
        for i in text:
            clean_text = clean_text + ' ' + i
        
        text_tokens = word_tokenize(clean_text)
        tokens_without_sw = [word for word in text_tokens if not word in stopwords.words('english')]
        
        coherence_val = 0
        for i in range(len(tokens_without_sw) - 1):
            coherence_val += cooccur_df[tokens_without_sw[i]][tokens_without_sw[i+1]]
            
        coherence_val /= (len(tokens_without_sw)-1)
        return [coherence_val]

    def tournament(self,Population,m,best):
        from_best = len(best)
        # print("best:",best)
        except_best = m - from_best
        # df = pd.DataFrame()
        #selected_polulation = pd.DataFrame(columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError","word_array"])
        ######################### when no syllables rule
        # selected_polulation = Population.iloc[:nn]
        # random_number = random.sample(list(Population.iloc[nn:].index),p)
        # Population.sort_values(by=['GrammarError'], inplace=True)
        # Population2 = Population[~Population.isin(selected_polulation).all(1)]
        ######################### when syllables rule
        if len(best) >0:
            selected_polulation = []
            words_array_selected = []
            for b in best:
                c = Population.iloc[b].values[:-1]
                d = Population.iloc[b].values[-1]
                selected_polulation.append(c)
                words_array_selected.append(d)
            words_array_selected = pd.DataFrame(dict(zip(["words_array"],[words_array_selected])))
            selected_polulation = pd.DataFrame(selected_polulation,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError","Coherence"])
            selected_polulation = pd.concat([selected_polulation,words_array_selected],axis=1)
            # print("poems selected by user")
            # display(selected_polulation)
            Population2 = Population[~Population.isin(selected_polulation).all(1)]
            Population2 = Population2.sort_values(by=['GrammarError'])
            selected_polulation2 = Population2[(Population2["Syllables1"] == "5")&(Population2["Syllables2"] == "6")&(Population2["Syllables3"] == "6")&(Population2["Syllables4"] == "5")]
            # selected_polulation2 = Population2[(Population2["Syllables1"] >= "3")]
            # print("grammar sorted poems by syll")
            # display(selected_polulation2)
            from_sy = len(selected_polulation2.index)
            if from_sy >= int(except_best/2):
                from_sy = int(except_best/2)
            else:
                from_sy = from_sy
            from_random = except_best - from_sy
            selected_polulation2 = selected_polulation2.iloc[:from_sy]
            selected_polulation = pd.concat([selected_polulation,selected_polulation2],axis=0)
            selected_polulation = selected_polulation.reset_index(drop=True)
            display(selected_polulation)
            display(Population2)
            Population2 = Population2[~Population2.isin(selected_polulation).all(1)]
            Population2 = Population2.reset_index(drop=True)
            selected_polulation = pd.concat([selected_polulation,Population2.iloc[:from_random]],axis=0)
            selected_polulation = selected_polulation.reset_index(drop=True)
            # print("SELECTED")
            # display(selected_polulation)
            words_array_selected = list(selected_polulation["words_array"])
            # print("TYPE",type(selected_polulation["words_array"]),words_array_selected)
            return selected_polulation,words_array_selected
        else:
            Population2 = Population.sort_values(by=['GrammarError'])
            random = int(except_best/3)
            grammar = except_best - random
            selected_polulation = Population2[:grammar]
            Population2 = Population2[~Population2.isin(selected_polulation).all(1)]
            selected_polulation2 = Population2[:random]
            selected_polulation = pd.concat([selected_polulation,selected_polulation2],axis=0)
            selected_polulation = selected_polulation.reset_index(drop=True)
            # print("SELECTED")
            # display(selected_polulation)
            words_array_selected = list(selected_polulation["words_array"])
            # print("TYPE",type(selected_polulation["words_array"]),words_array_selected)
            return selected_polulation,words_array_selected



    def create_set_of_poems(self,n):
            poem_population = []
            syllables_count = []
            grammar_error = []
            words_array_population = []
            coherence_arr = []
            for i in range(n):
                words_array = self.select_random_words(words_df)
                poem = self.create_poems(words_array)
                sum_syllables = self.store_syllables(words_array)
                poem_error = self.Grammar_checker(poem)
                coherence = self.coherence_checker(poem)
                poem_population.append(poem)
                syllables_count.append(sum_syllables)
                grammar_error.append(poem_error)
                coherence_arr.append(coherence)
                words_array_population.append(words_array)
            words_array2 = pd.DataFrame(dict(zip(["words_array"],[words_array_population])))
            Final = np.hstack((poem_population,syllables_count,grammar_error,coherence_arr))
            Population_data = pd.DataFrame(Final,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError","Coherence"])
            Pop_data = pd.concat([Population_data,words_array2],axis=1)
            # print()
            # print("PARENTS")
            # display(Pop_data)

            return Pop_data

    def child_poems(self,words_array,cross):
        child_population = []
        syllables_count = []
        grammer_error = []
        words_array_child = []
        coherence_arr = []
        # print("words_array",words_array)
        # print(type(words_array))
        if len(words_array)%2 == 2:
            for i in range(0,len(words_array)-1):
                child1,child2,child1_poem,child2_poem = self.crossover(words_array[i],words_array[i+1],cross)
                child_population.append(child1_poem)
                child_population.append(child2_poem)
                sum_syllables1 = self.store_syllables(child1)
                sum_syllables2 = self.store_syllables(child2)
                poem_error1 = self.Grammar_checker(child1_poem)
                poem_error2 = self.Grammar_checker(child2_poem)
                coherence1 = self.coherence_checker(child1_poem)
                coherence2 = self.coherence_checker(child2_poem)
                syllables_count.append(sum_syllables1)
                syllables_count.append(sum_syllables2)
                grammer_error.append(poem_error1)
                grammer_error.append(poem_error2)
                coherence_arr.append(coherence1)
                coherence_arr.append(coherence2)
                words_array_child.append(child1)
                words_array_child.append(child2)
        else:
            for i in range(0,len(words_array)-1,2):
                # print(words_array[i],words_array[i+1])
                child1,child2,child1_poem,child2_poem = self.crossover(words_array[i],words_array[i+1],cross)
                child_population.append(child1_poem)
                child_population.append(child2_poem)
                sum_syllables1 = self.store_syllables(child1)
                sum_syllables2 = self.store_syllables(child2)
                poem_error1 = self.Grammar_checker(child1_poem)
                poem_error2 = self.Grammar_checker(child2_poem)
                coherence1 = self.coherence_checker(child1_poem)
                coherence2 = self.coherence_checker(child2_poem)
                syllables_count.append(sum_syllables1)
                syllables_count.append(sum_syllables2)
                grammer_error.append(poem_error1)
                grammer_error.append(poem_error2)
                coherence_arr.append(coherence1)
                coherence_arr.append(coherence2)
                words_array_child.append(child1)
                words_array_child.append(child2)
        words_array_child = pd.DataFrame(dict(zip(["words_array"],[words_array_child])))
        Final1 = np.hstack((child_population,syllables_count,grammer_error,coherence_arr))
        child_data = pd.DataFrame(Final1,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError","Coherence"])
        child_data = pd.concat([child_data,words_array_child],axis=1)
        child_data = child_data.reset_index(drop=True)
        print()
        # print("CHILD")
        # display(child_data)
        return child_data

    def selection_from_child_parents(self,x,child_data,selected_Pop_data):
        reduce = (len(list(child_data.index)) + len(list(selected_Pop_data.index))) - x
        n = int(reduce/3)
        p = reduce - n
        if reduce > len(list(child_data.index)) + len(list(selected_Pop_data.index)):
            print("ERROR")
        else:
            if n > len(child_data):
                n = len(child_data)
                p = reduce - p
            if n == 0:
                n = 1
                p = 1
        d = pd.concat([selected_Pop_data,child_data],axis=0)
        d = d.reset_index(drop=True)
        # print("p:",p)
        # print("n:",n)
        random_number1 = random.sample(list(selected_Pop_data.index),p)
        random_number2 = random.sample(list(range(len(list(selected_Pop_data.index)),len(list(d.index)))),n)
        random_number1.extend(random_number2)
        # print(random_number1)
        for i in random_number1:
          d.drop(i,axis=0,inplace = True)
        d = d.reset_index(drop=True)
        # print("NEW POPULATION")
        # display(d)
        r = d[["Line1","Line2","Line3","Line4"]]
        return d,r
    
    def __init__(self,words_df,n,cross,epoch,z):
        Selection_number = int(round(0.85*n))
        best = []
        assert n>1 ,f"Population {n} cannot be less than or equal to 1"
        # assert n>=Selection_number, f"Cannot select {Selection_number} poems from total population of {n}"
        assert cross>1, f"Minimum number for crossover should be 1"
        # assert 2*n >= Selection_number ,f"Cannot make initial population of {n} if only {Selection_number} parents are selected"

        Population_data = self.create_set_of_poems(n)
        for i in range(epoch):
            Selected_Population_data,selected_polulation_word_array = self.tournament(Population_data,Selection_number,best)
            child_population = self.child_poems(selected_polulation_word_array,cross)
            final_population,final_poems = self.selection_from_child_parents(n,child_population,Population_data)
            if i != epoch-1:
                if (i+1)%10 == 0: # User feedback every 10 epochs
                    print("Select best poems")
                    display(final_poems)
                    best = (list(map(int,input("\n Enter best poems numbers: ").strip().split())))[:z]
                Population_data = final_population
                print("Final poems")
                display(final_population)

            else:
                print("###################################################")
                print("Last Poems Standing")
                print("###################################################")
                display(final_population)

In [10]:
#4 (words_df,intial_population,crossover,epochs,user_selection_poems_number):
poem = poem_generation(words_df,20,2,12,5)


Final poems


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,Coherence,words_array
0,are soar fear over,one everyone too never,undressed little there gloom,on one life poets,6,9,8,5,17,2.4833185773823663e-05,"[are, soar, fear, over, one, everyone, too, never, undressed, little, there, gloom, on, one, life, poets]"
1,be footfalls its to,with is treat we,light with so our,of and looking all,5,4,4,5,13,0.0,"[be, footfalls, its, to, with, is, treat, we, light, with, so, our, of, and, looking, all]"
2,perspiration of the must,glory indents lack on,wrong all we doors,imagination of the your,7,6,4,8,33,0.0,"[perspiration, of, the, must, glory, indents, lack, on, wrong, all, we, doors, imagination, of, the, your]"
3,nights death spirited we,no observed beams breeze,was they hope in,from enough the knell,6,7,4,5,25,0.0,"[nights, death, spirited, we, no, observed, beams, breeze, was, they, hope, in, from, enough, the, knell]"
4,year your adrift fire,is brush therapy remembering,we primordial of are,window to of cars,5,9,7,5,14,0.0,"[year, your, adrift, fire, is, brush, therapy, remembering, we, primordial, of, are, window, to, of, cars]"
5,all its from i,adrift sing winters this,natures and it children,a stay expressed now,4,6,7,6,17,1.7995062154944685e-06,"[all, its, from, i, adrift, sing, winters, this, natures, and, it, children, a, stay, expressed, now]"
6,others mortal them colored,sailing on can us,its to always of,from they our undermining,8,5,5,7,20,0.0,"[others, mortal, them, colored, sailing, on, can, us, its, to, always, of, from, they, our, undermining]"
7,out be someone sorrowful,all human beacon death,of for are home,ever life our the,8,6,5,5,12,7.712169494976294e-06,"[out, be, someone, sorrowful, all, human, beacon, death, of, for, are, home, ever, life, our, the]"
8,was as live or,move heart devoured incompetent,everything lift outside each,we flapping crafted inside,4,9,9,8,19,0.0,"[was, as, live, or, move, heart, devoured, incompetent, everything, lift, outside, each, we, flapping, crafted, inside]"
9,out be someone sorrowful,all that much it,and our glow to,her away death of,8,4,4,5,12,0.0,"[out, be, someone, sorrowful, all, that, much, it, and, our, glow, to, her, away, death, of]"



Final poems


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,Coherence,words_array
0,are soar fear over,one everyone too never,undressed little there gloom,on one life poets,6,9,8,5,17,2.4833185773823663e-05,"[are, soar, fear, over, one, everyone, too, never, undressed, little, there, gloom, on, one, life, poets]"
1,be footfalls its to,with is treat we,light with so our,of and looking all,5,4,4,5,13,0.0,"[be, footfalls, its, to, with, is, treat, we, light, with, so, our, of, and, looking, all]"
2,perspiration of the must,glory indents lack on,wrong all we doors,imagination of the your,7,6,4,8,33,0.0,"[perspiration, of, the, must, glory, indents, lack, on, wrong, all, we, doors, imagination, of, the, your]"
3,all its from i,adrift sing winters this,natures and it children,a stay expressed now,4,6,7,6,17,1.7995062154944685e-06,"[all, its, from, i, adrift, sing, winters, this, natures, and, it, children, a, stay, expressed, now]"
4,out be someone sorrowful,all human beacon death,of for are home,ever life our the,8,6,5,5,12,7.712169494976294e-06,"[out, be, someone, sorrowful, all, human, beacon, death, of, for, are, home, ever, life, our, the]"
5,out be someone sorrowful,all that much it,and our glow to,her away death of,8,4,4,5,12,0.0,"[out, be, someone, sorrowful, all, that, much, it, and, our, glow, to, her, away, death, of]"
6,the let gloom on,one life poets over,one everyone too never,undressed little there to,4,6,9,8,18,2.759242863758185e-05,"[the, let, gloom, on, one, life, poets, over, one, everyone, too, never, undressed, little, there, to]"
7,fear and eyes we,so air and through,see band hope love,on because the in,4,4,4,6,11,8.997531077472342e-06,"[fear, and, eyes, we, so, air, and, through, see, band, hope, love, on, because, the, in]"
8,feeling indents shined others,we now no grew,every as the winters,all treat was knowledge,8,4,7,6,17,0.0,"[feeling, indents, shined, others, we, now, no, grew, every, as, the, winters, all, treat, was, knowledge]"
9,love on because the,in all that much,it and and through,see band hope our,6,4,4,4,11,0.0,"[love, on, because, the, in, all, that, much, it, and, and, through, see, band, hope, our]"



Final poems


...

,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,Coherence,words_array
0,band fear and eyes,we away death of,out be someone sorrowful,all out be always,4,5,8,5,12,0.0,"[band, fear, and, eyes, we, away, death, of, out, be, someone, sorrowful, all, out, be, always]"
1,band fear and eyes,we away death away,death of out be,to so air and,4,6,4,4,13,0.0,"[band, fear, and, eyes, we, away, death, away, death, of, out, be, to, so, air, and]"
2,ever that it and,away poets see band,as the the eyes,over her grew every,5,5,4,7,14,3.0848677979905172e-06,"[ever, that, it, and, away, poets, see, band, as, the, the, eyes, over, her, grew, every]"
3,footfalls her grew every,one fear over one,ever that it and,as the the eyes,7,7,5,4,18,0.0,"[footfalls, her, grew, every, one, fear, over, one, ever, that, it, and, as, the, the, eyes]"
4,through see band fear,and eyes we away,death of and eyes,now no grew every,4,5,4,6,18,0.0,"[through, see, band, fear, and, eyes, we, away, death, of, and, eyes, now, no, grew, every]"
5,away death of out,be to so air,death of band fear,our love on and,5,4,4,4,14,5.398518646483404e-06,"[away, death, of, out, be, to, so, air, death, of, band, fear, our, love, on, and]"
6,out be always hope,love on because the,death of as the,the eyes over be,5,6,4,5,15,1.3496296616208513e-05,"[out, be, always, hope, love, on, because, the, death, of, as, the, the, eyes, over, be]"
7,and away poets see,band light be ever,that it as the,the eyes over her,5,5,4,5,14,0.0,"[and, away, poets, see, band, light, be, ever, that, it, as, the, the, eyes, over, her]"
8,through see band fear,and someone and through,too never undressed or,move heart devoured incompetent,4,6,7,9,17,0.0,"[through, see, band, fear, and, someone, and, through, too, never, undressed, or, move, heart, devoured, incompetent]"
9,ever that it and,away poets see band,as the the band,fear death away death,5,5,4,5,14,1.1996708103296456e-05,"[ever, that, it, and, away, poets, see, band, as, the, the, band, fear, death, away, death]"


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,Coherence,words_array
0,band fear and eyes,we away death away,death of out be,to so air and,4,6,4,4,13,0.0,"[band, fear, and, eyes, we, away, death, away, death, of, out, be, to, so, air, and]"
1,ever that it and,away poets see band,as the the eyes,over her grew every,5,5,4,7,14,3.0848677979905172e-06,"[ever, that, it, and, away, poets, see, band, as, the, the, eyes, over, her, grew, every]"
2,footfalls her grew every,one fear over one,ever that it and,as the the eyes,7,7,5,4,18,0.0,"[footfalls, her, grew, every, one, fear, over, one, ever, that, it, and, as, the, the, eyes]"
3,through see band fear,and eyes we away,death of and eyes,now no grew every,4,5,4,6,18,0.0,"[through, see, band, fear, and, eyes, we, away, death, of, and, eyes, now, no, grew, every]"
4,away death of out,be to so air,death of band fear,our love on and,5,4,4,4,14,5.398518646483404e-06,"[away, death, of, out, be, to, so, air, death, of, band, fear, our, love, on, and]"


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,Coherence,words_array
18,be ever that it,as the the eyes,over her see band,as the the and,5,4,5,4,10,0.0,"[be, ever, that, it, as, the, the, eyes, over, her, see, band, as, the, the, and]"
11,we away death of,out be someone its,to in our the,with our love on,5,6,4,4,11,0.0,"[we, away, death, of, out, be, someone, its, to, in, our, the, with, our, love, on]"
0,band fear and eyes,we away death of,out be someone sorrowful,all out be always,4,5,8,5,12,0.0,"[band, fear, and, eyes, we, away, death, of, out, be, someone, sorrowful, all, out, be, always]"
16,to so air and,that it and over,her grew every one,fear over one ever,4,5,7,7,13,0.0,"[to, so, air, and, that, it, and, over, her, grew, every, one, fear, over, one, ever]"
15,as the the eyes,band fear and eyes,we away death away,death of out be,4,4,6,4,13,0.0,"[as, the, the, eyes, band, fear, and, eyes, we, away, death, away, death, of, out, be]"
13,we light be its,to in our the,with our love away,death of out be,4,4,5,4,13,7.557926105076768e-05,"[we, light, be, its, to, in, our, the, with, our, love, away, death, of, out, be]"
10,of out be to,so air and and,eyes we away grew,every eyes over her,4,4,5,7,13,0.0,"[of, out, be, to, so, air, and, and, eyes, we, away, grew, every, eyes, over, her]"
19,death of out be,to because the death,of as the the,eyes over be so,4,6,4,5,13,0.0,"[death, of, out, be, to, because, the, death, of, as, the, the, eyes, over, be, so]"
1,band fear and eyes,we away death away,death of out be,to so air and,4,6,4,4,13,0.0,"[band, fear, and, eyes, we, away, death, away, death, of, out, be, to, so, air, and]"
5,away death of out,be to so air,death of band fear,our love on and,5,4,4,4,14,5.398518646483404e-06,"[away, death, of, out, be, to, so, air, death, of, band, fear, our, love, on, and]"



###################################################
Last Poems Standing
###################################################


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,Coherence,words_array
0,band fear and eyes,we away death of,out be someone sorrowful,all out be always,4,5,8,5,12,0.0,"[band, fear, and, eyes, we, away, death, of, out, be, someone, sorrowful, all, out, be, always]"
1,band fear and eyes,we away death away,death of out be,to so air and,4,6,4,4,13,0.0,"[band, fear, and, eyes, we, away, death, away, death, of, out, be, to, so, air, and]"
2,through see band fear,and eyes we away,death of and eyes,now no grew every,4,5,4,6,18,0.0,"[through, see, band, fear, and, eyes, we, away, death, of, and, eyes, now, no, grew, every]"
3,out be always hope,love on because the,death of as the,the eyes over be,5,6,4,5,15,1.3496296616208513e-05,"[out, be, always, hope, love, on, because, the, death, of, as, the, the, eyes, over, be]"
4,and away poets see,band light be ever,that it as the,the eyes over her,5,5,4,5,14,0.0,"[and, away, poets, see, band, light, be, ever, that, it, as, the, the, eyes, over, her]"
5,ever that it and,away poets see band,as the the band,fear death away death,5,5,4,5,14,1.1996708103296456e-05,"[ever, that, it, and, away, poets, see, band, as, the, the, band, fear, death, away, death]"
6,because the fear and,eyes so air and,through see band and,treat we light be,6,4,4,4,23,0.0,"[because, the, fear, and, eyes, so, air, and, through, see, band, and, treat, we, light, be]"
7,someone and eyes we,away death of out,be band fear its,from never all natures,6,5,4,7,17,0.0,"[someone, and, eyes, we, away, death, of, out, be, band, fear, its, from, never, all, natures]"
8,death of out be,to because the death,of as the the,eyes over be so,4,6,4,5,13,0.0,"[death, of, out, be, to, because, the, death, of, as, the, the, eyes, over, be, so]"
9,ever that band fear,eyes over her grew,every be to so,air and it and,5,5,6,4,16,0.0,"[ever, that, band, fear, eyes, over, her, grew, every, be, to, so, air, and, it, and]"


In [11]:
print(list(range(5,8)))

[5, 6, 7]


In [12]:
a = [2,3,4,5]

for i in a:
    print(i)

2
3
4
5
